In [ ]:
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM

# config = PeftConfig.from_pretrained("rajeev-dw9/med_llama_chat")
# print(config)
# model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
# print(model)
# print("---------------------------------------------------------")
# model = PeftModel.from_pretrained(model, "rajeev-dw9/med_llama")
# print(model)

In [ ]:
import pandas as pd

df = pd.read_csv("/home/ubuntu/Project_Files/Finetune/Data/sentences.csv")

def split_sentence(sentence):
    words = sentence.split()
    first_part = ' '.join(words[:-1])
    last_word = words[-1]
    return first_part, last_word

df['text'], df['pred'] = zip(*df['Sentence'].map(split_sentence))
df = df.drop(columns=['Sentence'])
df = df.head(100)
df.size

data = df
print(data)



In [ ]:
# import pandas as pd
# from sklearn.metrics import f1_score
# from transformers import pipeline

# # Assuming your data is in a CSV file named "data.csv"
# # data = pd.read_csv("data.csv")
# data = data.head(10)

# generator = pipeline('text-generation', model='rajeev-dw9/med_llama')

# def get_prediction(text):
#     print(text)
#     generated_text = generator(text, max_length=50, num_return_sequences=1)
#     pred = generated_text[0]['generated_text'].split(text)[-1].strip()
#     print(pred)
#     return pred

# # Apply the prediction function to your text data
# data['model_pred'] = data['text'].apply(get_prediction)

# f1 = f1_score(data['pred'], data['model_pred'], average='weighted')  # You can change the averaging method

# print(f"F1 Score: {f1}")


In [ ]:
import pandas as pd

df = pd.read_csv("/home/ubuntu/Project_Files/Finetune/Data/sentences.csv")

def split_sentence(sentence):
    words = sentence.split()
    first_part = ' '.join(words[:-1])
    last_word = words[-1]
    return first_part, last_word

df['text'], df['pred'] = zip(*df['Sentence'].map(split_sentence))
df = df.drop(columns=['Sentence'])
df = df.head(100)
df.size

data = df
print(data)



In [5]:
import pandas as pd
import torch
from sklearn.metrics import f1_score
from transformers import pipeline

data = data.head(10)

generator = pipeline('text-generation', model='rajeev-dw9/med_llama')

def create_prompt(text):
    prompt_template = """
    {}. What gene/protein is realted to this?
    """
    return prompt_template.format(text)
def get_prediction(text):
    prompt = create_prompt(text)
    print("|||||||||",prompt,"|||||||||") 
    generated_text = generator(prompt, max_length=200, num_return_sequences=1)
    pred = generated_text[0]['generated_text'].split(prompt)[-1].strip()  # Adjust this if necessary
    print(pred)
    print("|||||||||-----------------------------|||||||||")
    return pred

# Apply the prediction function to your text data
data['model_pred'] = data['text'].apply(get_prediction)
f1 = f1_score(data['pred'], data['model_pred'], average='weighted')  # You can change the averaging method

print(f"F1 Score: {f1}")

In the PI4KA, which is a type of
|||||||||-----------------------------|||||||||
||||||||| 
    In the SLC15A1, which is a type of gene/protein, there is a noted ppi of the gene/protein. What gene/protein is realted to this?
     |||||||||
In the SLC15A1,
|||||||||-----------------------------|||||||||
||||||||| 
    In the EIF3I, which is a type of gene/protein, there is a noted ppi of the gene/protein. What gene/protein is realted to this?
     |||||||||


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
from sklearn.metrics import f1_score
from transformers import pipeline

# Load your data
# data = pd.read_csv("data.csv")
data = data.head(10)  # For demonstration, using only the first 10 entries

# Check for available GPUs and prepare device IDs
available_gpus = torch.cuda.device_count()  # Count available GPUs
devices = list(range(available_gpus))  # List of device IDs

# Function to create a pipeline for a specific device
def create_pipeline(device):
    return pipeline('text-generation', model='rajeev-dw9/med_llama', device=device)

# Create a pipeline for each available GPU
pipelines = [create_pipeline(device) for device in devices]

def create_prompt(text):
    prompt_template = """
    "Given this information: {}. Based on current research and databases, the noted interaction partner for this gene/protein is: "
    """
    return prompt_template.format(text)


def get_predictions(data, pipelines):
    preds = []
    for i, row in data.iterrows():
        pipeline = pipelines[i % len(pipelines)]  # Distribute data across pipelines
        prompt = create_prompt(row['text'])
        generated_text = pipeline(prompt, max_length=100, num_return_sequences=1)
        pred = generated_text[0]['generated_text'].split(prompt)[-1].strip()
        preds.append(pred)
    return preds

data['model_pred'] = get_predictions(data, pipelines)
f1 = f1_score(data['pred'], data['model_pred'], average='weighted')  # You can change the averaging method

print(f"F1 Score: {f1}")
